In [2]:
!pip install torch

In [3]:
!pip install accelerate~=0.20
!pip install transformers~=4.30.2

## Sentiment Analysis

In [4]:
import pandas as pd

In [5]:
from transformers import TextClassificationPipeline, AutoModelForSequenceClassification, AutoTokenizer

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_name = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 3)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, max_length=64, truncation=True, padding = 'max_length')

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [9]:
list_subs = ['bitcoin','bitcoinbeginners','bitcoinmarkets','bitcoinmining','btc']

In [11]:
for subreddit in list_subs:
    final_df = pd.read_csv(f"Niks/Data/Cleaned_Data/crypto_bitcoin/{subreddit}_submission.csv")
    reddit_texts = final_df['selftext'].tolist()
    preds = pipe(reddit_texts)
    for element in preds:
        label = element['label']
        if label == "Bearish":
            element['label']="Negative"
        elif label == 'Bullish':
            element['label']="Positive"
        else:
            element['label']="Neutral"
    new_data_df = pd.DataFrame(preds)
    print(f"{subreddit} scores min: ",new_data_df.score.min(),"max: ",new_data_df.score.max())
    labeled_df = pd.concat([final_df, new_data_df], axis=1)
    labeled_df.to_csv(f"Niks/Data/Sentiment/crypto_bitcoin/{subreddit}_submission_sentiment.csv",index=False)

bitcoin scores min:  0.33553192019462585 max:  0.9943163990974426
bitcoinbeginners scores min:  0.34520962834358215 max:  0.9958756566047668
bitcoinmarkets scores min:  0.4563077688217163 max:  0.9896363615989685
bitcoinmining scores min:  0.3491261601448059 max:  0.991299569606781
btc scores min:  0.3407171964645386 max:  0.9951267242431641
